# Essential Libraries

In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2, csv
import pandas as pd
import numpy as np
import random
import splitfolders
import shutil
import os
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout, BatchNormalization, Input
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras import regularizers
from tensorflow.keras.models import load_model

2025-06-30 22:08:01.834027: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("jessicali9530/stanford-dogs-dataset")

print("Path to dataset files:", path) # <== This is the path to the dataset files

### Cleaning & Pre-processing

In [4]:
ImagesDir = "/Users/muhammadabdelmohsen/Desktop/MasterML/DogBreedsClassification/Images"
trainDir = "/Users/muhammadabdelmohsen/Desktop/MasterML/DogBreedsClassification/train"
testDir = "/Users/muhammadabdelmohsen/Desktop/MasterML/DogBreedsClassification/test"
valDir = "/Users/muhammadabdelmohsen/Desktop/MasterML/DogBreedsClassification/val"

# Remove the IDs

In [ ]:
for i in os.listdir(ImagesDir):
    if i == ".DS_Store":
        continue
    name = i.split('-', 1)[1]   # Split at first dash and take the second part
    print(name)


In [5]:
Folders = set(os.listdir(ImagesDir))
print("Unique Dog Breeds Found:", len(Folders))
Folders

Unique Dog Breeds Found: 121


{'.DS_Store',
 'Afghan_hound',
 'African_hunting_dog',
 'Airedale',
 'American_Staffordshire_terrier',
 'Appenzeller',
 'Australian_terrier',
 'Bedlington_terrier',
 'Bernese_mountain_dog',
 'Blenheim_spaniel',
 'Border_collie',
 'Border_terrier',
 'Boston_bull',
 'Bouvier_des_Flandres',
 'Brabancon_griffon',
 'Brittany_spaniel',
 'Cardigan',
 'Chesapeake_Bay_retriever',
 'Chihuahua',
 'Dandie_Dinmont',
 'Doberman',
 'English_foxhound',
 'English_setter',
 'English_springer',
 'EntleBucher',
 'Eskimo_dog',
 'French_bulldog',
 'German_shepherd',
 'German_short-haired_pointer',
 'Gordon_setter',
 'Great_Dane',
 'Great_Pyrenees',
 'Greater_Swiss_Mountain_dog',
 'Ibizan_hound',
 'Irish_setter',
 'Irish_terrier',
 'Irish_water_spaniel',
 'Irish_wolfhound',
 'Italian_greyhound',
 'Japanese_spaniel',
 'Kerry_blue_terrier',
 'Labrador_retriever',
 'Lakeland_terrier',
 'Leonberg',
 'Lhasa',
 'Maltese_dog',
 'Mexican_hairless',
 'Newfoundland',
 'Norfolk_terrier',
 'Norwegian_elkhound',
 'Norwic

In [ ]:
# add all the folders in the foldersSet to the trainDir, testDir, valDir
if not os.path.exists(trainDir):
    os.makedirs(trainDir)
if not os.path.exists(testDir):
    os.makedirs(testDir)
if not os.path.exists(valDir):
    os.makedirs(valDir)
# Create subdirectories for train, test, and validation sets dog breeds
for folder in Folders:

    if folder == ".DS_Store":
        continue
    folderPath = os.path.join(ImagesDir, folder)
    if not os.path.exists(os.path.join(trainDir, folder)):
        os.makedirs(os.path.join(trainDir, folder))
    if not os.path.exists(os.path.join(testDir, folder)):
        os.makedirs(os.path.join(testDir, folder))
    if not os.path.exists(os.path.join(valDir, folder)):
        os.makedirs(os.path.join(valDir, folder))


In [17]:
random.seed(42)

for breed in os.listdir(ImagesDir):
    breed_path = os.path.join(ImagesDir, breed)
    if breed == ".DS_Store":
        continue

    images = [f for f in os.listdir(breed_path) if os.path.isfile(os.path.join(breed_path, f))]
    random.shuffle(images)

    nTotal = len(images)
    nTrain = int(0.8 * nTotal)
    nVal = int(0.1 * nTotal)
    nTest = nTotal - nTrain - nVal

    train_files = images[:nTrain]
    val_files = images[nTrain:nTrain + nVal]
    test_files = images[nTrain + nVal:]

    for split_name, split_files in zip(
        ["train", "val", "test"],
        [train_files, val_files, test_files]
    ):
        split_breed_dir = os.path.join("/Users/muhammadabdelmohsen/Desktop/ML & DL & LLM Projects/DogBreedsClassification", split_name, breed)
        os.makedirs(split_breed_dir, exist_ok=True)

        for file in split_files:
            src = os.path.join(breed_path, file)
            dst = os.path.join(split_breed_dir, file)
            shutil.copy2(src, dst)

    print(f"Breed: {breed} -> Train: {nTrain}, Val: {nVal}, Test: {nTest}")

print("✅ Done splitting!")

# Data Augmentation

In [20]:
trainData = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
testData = ImageDataGenerator(rescale=1./255)
valData = ImageDataGenerator(rescale=1./255)

In [21]:
trainData = trainData.flow_from_directory(
    trainDir,
    target_size=(96, 96),
    color_mode='grayscale',
    batch_size=32,
    class_mode='categorical',
    shuffle=True
)

testData = testData.flow_from_directory(
    testDir,
    target_size=(96, 96),
    color_mode='grayscale',
    batch_size=32,
    class_mode='categorical',
    shuffle=True
)

valData = valData.flow_from_directory(
    valDir,
    target_size=(96, 96),
    color_mode='grayscale',
    batch_size=32,
    class_mode='categorical',
    shuffle=True
)

Found 16418 images belonging to 120 classes.
Found 2153 images belonging to 120 classes.
Found 2009 images belonging to 120 classes.


In [ ]:

l2_strength = 0.01 

model = Sequential()

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=(96, 96, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(256, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(512, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(1024, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Flatten())

model.add(Dense(256, activation='relu', kernel_regularizer=regularizers.l2(l2_strength)))
model.add(Dense(120, activation='softmax'))  # Output layer



/Users/muhammadabdelmohsen/Desktop/MasterML/ML_NN_Env/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
earlyStopping = EarlyStopping(patience = 5, restore_best_weights=True, monitor=['val_loss'])
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)

In [27]:
history = model.fit(
    trainData,
    validation_data=valData,
    epochs=2,
    callbacks=[earlyStopping],
    verbose=1
)

Epoch 1/2
116/514 ━━━━━━━━━━━━━━━━━━━━ 7:56 1s/step - accuracy: 0.0097 - loss: 9.1185

KeyboardInterrupt: 